# Data Information:

## Input variables:


## Output variable (desired target):


# Data preprocessing 

Import libraries and data, select a small sample set for trainning


In [214]:
from sklearn import datasets
import numpy as np
import pandas as pd
import copy

lol_origin = pd.read_csv('./leagueoflegends/leagueoflegends.csv')
lol = lol_origin.copy()

In [215]:
for x in ['golddiff','goldblueTop','goldblueJungle','goldblueMiddle','goldblueADC','goldblueSupport',
          'goldredTop','goldredJungle','goldredMiddle','goldredADC','goldredSupport']:
    lol[x] = [int(y.split(',')[14]) for y in lol_origin[x]]

for x in ['Top','Jungle','Middle','ADC','Support']:
    lol['golddiff' + x ] = lol['goldblue' + x ] - lol['goldred' + x ]
    del lol['goldblue' + x ]
    del lol['goldred' + x ]

In [216]:
def get_data15(feature):
    info = [x[2:-2] for x in feature]
    data15 = []
    for y in info:
        y = y.split('], [')
        j = 0
        for z in y:
            if z.strip() =='':
                break
            else:
                z=z.split(',')
                n =float(z[0])
                if n < 15.0 :
                    j = j + 1
        data15.append(j)
    return data15

In [217]:
for x in ['bKills','bTowers','bInhibs','bDragons','bHeralds',
          'rKills','rTowers','rInhibs','rDragons','rHeralds']:
    del lol[x]
    lol[x] = get_data15(lol_origin[x])

In [235]:
champ_names = lol['blueTopChamp']
for x in ['blueMiddleChamp','blueJungleChamp','blueADCChamp','blueSupportChamp',
          'redTopChamp','redMiddleChamp','redJungleChamp','redADCChamp','redSupportChamp']:
    champ_names = champ_names.append(lol[x])
len(champ_names.unique())

140

### Processing the ordinal and nominal data,seperately

In my opinion, only 'player' variable is ordinal


In [236]:
from sklearn.preprocessing import LabelEncoder
champ_le = LabelEncoder()
champ_le.fit_transform(champ_names)
lol['blueTopChamp'] = champ_le.transform(lol_origin['blueTopChamp'])
lol['redTopChamp'] = champ_le.transform(lol_origin['redTopChamp'])
lol.loc[:,['blueTopChamp','redTopChamp']]

,blueTopChamp,redTopChamp
0,40,34
1,34,40
2,90,100
3,40,34
4,34,100
5,51,34
6,40,90
7,90,93
8,100,51
9,40,34


### Splitting data into 70% training and 30% test data 
### Bringing features onto the same scale

Here we choose standardization instead of normalization

Standardizing both the numeric and the Categorical features

<br>
<br>